In [1]:
import sys
sys.path.append('../')

import numpy as np
from scipy import ndimage

from skimage.filters import sobel_h
from skimage.filters import sobel_v
from scipy import stats


import os
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.proj3d import proj_transform
from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.mplot3d import proj3d


import scienceplots
from tensorflow.python.client import device_lib

#plt.rcParams['figure.figsize'] = [10,10]

import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import decode_predictions
from tensorflow.keras.applications import VGG16

from tensorflow.nn import depthwise_conv2d
from tensorflow.math import multiply, reduce_sum, reduce_mean,reduce_euclidean_norm, sin, cos, abs
from tensorflow import stack, concat, expand_dims

import tensorflow_probability as tfp

from utils.utils import *
import cv2

from scipy import ndimage, fft
from io import BytesIO

import imageio.v2 as imageio

plt.style.use(['science', 'ieee'])
plt.rcParams.update({'figure.dpi': '100'})

/tmp/ipykernel_32715/3418332839.py:5: UserWarning: A NumPy version >=1.23.5 and <2.5.0 is required for this version of SciPy (detected version 1.23.0)
  from scipy import ndimage
2025-12-01 13:56:22.275931: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-01 13:56:22.442387: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-12-01 13:56:22.442461: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-12-01 13:56:22.467045: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to reg

In [2]:
k = 3   # kernel size
beta2s = [1, 0, 0.25, 0.5, 0.75] #[0, 0.25, 0.75, 1]  
activations = [tf.nn.relu]
timestamps = [1,5,10]
experiment_name = "unipolar_single_pixel"
box_dims = [20, 16]
step =   2 # Plot axis step

In [3]:
tf.nn.relu.__name__

'relu'

In [4]:
# Single pixel input

img = np.zeros((115,115)) # cv2.imread('input4.png', 0)/255. 
mid = img.shape[0]//2
img[mid, mid] = 1.
print(img.shape)

(115, 115)


In [5]:
filters = np.zeros((3,3,1,1))
img = tf.cast(tf.repeat(tf.expand_dims([img], axis=-1) , repeats = filters.shape[-2], axis=-1), dtype=tf.float32) 


2025-12-01 13:56:24.181186: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-12-01 13:56:24.349316: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-12-01 13:56:24.350652: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [6]:
import matplotlib.patches as mpatches



filters = np.zeros((3,3,1,1))
x = tf.cast(tf.repeat(tf.expand_dims([img], axis=-1) , repeats = filters.shape[-2], axis=-1), dtype=tf.float32) 



#Get image height and width

vals_x = img[0, img.shape[1]//2, :, :]
vals_y = img[0, :, img.shape[2]//2, :]
nonzeros = np.nonzero(vals_x)[0]
if nonzeros.size > 0:
    length_y = nonzeros[-1] - nonzeros[0] + 1
else:
    length_y = 0  # all zeros
nonzeros = np.nonzero(vals_x)[0]
if nonzeros.size > 0:
    length_x = nonzeros[-1] - nonzeros[0] + 1
else:
    length_x = 0  # all zeros



#step = 2**np.floor(np.log2(( (length_x) + np.max(timestamps) )// 5))


max_x = int(box_dims[0]*(.7))
min_x = -int(box_dims[0]*(.3))

max_y = box_dims[1]//2
min_y = -(box_dims[1]//2)

print(max_y, min_y)
print(length_x, length_y)





for activation in activations:
    directory = f"figures/{experiment_name}_{activation.__name__}_{k}x{k}"
    if not os.path.exists(directory):
        os.makedirs(directory)
    for beta2 in beta2s:

        # list of frames (as numpy arrays) for the GIF
        frames = []

        filters = np.zeros((3,3,1,1))

        t = np.zeros((3,3))
        t[1, 0] = np.sqrt(beta2)
        t[0, 0] = np.sqrt(1-beta2)
        filters = np.reshape(fft.idctn(t, norm='ortho'), (3,3,1,1)) 

        w = tf.cast(filters, dtype=tf.float32)
        w = tf.transpose(w, perm=(1,0,2,3))

        x = img  # starting image

        # evolve over time
        for i in range(timestamps[-1] + 1):
            x = x / np.std(x)

            if i in timestamps:
                # --- create a single-frame figure ---
                fig, ax = plt.subplots(figsize=(6, 6))

                vals = x[0, x.shape[1]//2, :, :]
                vals = vals / np.sum(vals)

                pos = np.expand_dims(
                    np.linspace(-(x.shape[1]//2), x.shape[1]//2, x.shape[1]),
                    -1
                )
                mean = tf.reduce_sum(pos * vals)
                var = tf.reduce_sum(((pos - mean)**2) * vals)
                std = np.sqrt(var)

                im = ax.imshow(np.sum(x[0, :, :, :], axis=-1))
                ax.plot(mid, mid, marker='+', color='red')

                ax.annotate(r'$\sigma_x=$' + f"{std:.2f}",
                            (0.05, 0.01), ha='left', va='bottom',
                            color='white', xycoords='axes fraction', fontsize=20)
                ax.annotate(r'$\mu_x=$' + f"{mean:.2f}",
                            (0.05, 0.11), ha='left', va='bottom',
                            color='white', xycoords='axes fraction', fontsize=20)

                ax.set(
                    xlim=((mid + min_x, mid + max_x)),
                    ylim=((mid + min_y, mid + max_y))
                )

                # custom ticks
                xticks = np.arange(
                    np.sign(min_x) * (np.abs(min_x)//2) * 2,
                    step + (max_x//2) * 2,
                    step=step
                ) + mid
                yticks = np.arange(
                    np.sign(min_y) * (np.abs(min_y)//2) * 2,
                    step + (max_y//2) * 2,
                    step=step
                ) + mid

                ax.set_xticks(xticks)
                ax.set_yticks(yticks)
                ax.tick_params(axis='x', labelsize=15)
                ax.tick_params(axis='y', labelsize=15)

                ax.set_xticklabels([x_tick - mid for x_tick in xticks])
                ax.set_yticklabels([y_tick - mid for y_tick in yticks])

                if beta2 == 0:
                    ax.set_title(f"t = {i}", fontsize=25)

                fig.tight_layout()

                # ---- capture this frame into memory ----
                buf = BytesIO()
                fig.savefig(buf, format="png", dpi=fig.dpi, bbox_inches="tight")
                buf.seek(0)
                frame = imageio.imread(buf)
                frames.append(frame)

                plt.close(fig)

            # update x for next timestep
            x = activation(tf.nn.conv2d(x, w, strides=(1, 1), padding='SAME'))

        # ----- write the GIF for this (activation, beta2) -----
        gif_path = os.path.join(
            directory,
            f"{experiment_name}_{k}x{k}__{activation.__name__}_{beta2}.gif"
        )
        imageio.mimsave(gif_path, frames, duration=0.3)  # duration = sec/frame
        print(f"Saved GIF to {gif_path}")


2025-12-01 13:56:24.659252: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2025-12-01 13:56:24.747316: E external/local_xla/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

2025-12-01 13:56:24.748962: W external/local_xla/xla/stream_executor/gpu/asm_compiler.cc:225] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 12.0
2025-12-01 13:56:24.748973: W external/local_xla/xla/stream_executor/gpu/asm_compiler.cc:228] Used ptxas at ptxas
2025-12-01 13:56:24.749023: W external/local_xla/xla/stream_executor/gpu/redzone_allocator.cc:322] UNIMPLEMENTED: ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


8 -8
1 1


2025-12-01 13:56:24.981818: E external/local_xla/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

2025-12-01 13:56:24.983292: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2025-12-01 13:56:25.063158: E external/local_xla/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

2025-12-01 13:56:25.064356: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2025-12-01 13:56:25.065068: E external/local_xla/xla/str

Saved GIF to figures/unipolar_single_pixel_relu_3x3/unipolar_single_pixel_3x3__relu_1.gif
Saved GIF to figures/unipolar_single_pixel_relu_3x3/unipolar_single_pixel_3x3__relu_0.gif
Saved GIF to figures/unipolar_single_pixel_relu_3x3/unipolar_single_pixel_3x3__relu_0.25.gif
Saved GIF to figures/unipolar_single_pixel_relu_3x3/unipolar_single_pixel_3x3__relu_0.5.gif
Saved GIF to figures/unipolar_single_pixel_relu_3x3/unipolar_single_pixel_3x3__relu_0.75.gif


In [7]:
i

10

In [8]:
step

2